# Mongodb Aggregate And Group analysis


**1. avg**

**2. sum**

**3. project**

**Documentation:**
    
https://www.mongodb.com/docs/manual/reference/operator/aggregation/sum/

https://www.mongodb.com/docs/manual/reference/operator/aggregation/avg/

https://www.mongodb.com/docs/manual/reference/operator/aggregation/project/#mongodb-pipeline-pipe.-project

In [35]:
# from pymongo library import MongoClient module
from pymongo import MongoClient

In [36]:
# creation of object MongoClient (This MongoClient help up to connect to the database)
client = MongoClient()
    
# Connect with the portnumber and host  
client = MongoClient("mongodb://localhost:27017/")  

In [37]:
# Create a Database called as 'students' and Access database  
mydatabase = client['Students']  

In [38]:
# Create a Collection called as 'studentscores' inside students db and Access collection of the database  
collection = mydatabase['studentscores']

In [6]:
# Insert many documents inside the collection// Records in the form of JSON
data = [ 
    {"user":"Sid", "subject":"Database", "score":80}, 
    {"user":"Leo",  "subject":"JavaScript", "score":90}, 
    {"user":"Leo",  "title":"Database", "score":85}, 
    {"user":"Sid",  "title":"JavaScript", "score":75}, 
    {"user":"Leo",  "title":"Data Science", "score":60},
    {"user":"Sid",  "title":"Data Science", "score":95}] 
  
collection.insert_many(data)

# A. Group by SUM of total subjects

### Find Leo And Sid Total Subjects

In [39]:
#Here, we need to use AGGREGATE function (it comes with group by )
agg_result= collection.aggregate([
    { 
        "$group" :  
                  {"_id" : "$user",      # Here, we are GROUPING by user: '$user' is our field name and '_id' is a unique identity 
                   "Total Subject" : {"$sum" : 1}}    # $sum is going to work on with $user to find how many users are there!, Total Subject is a custom field that we are creating
    } 
])

In [40]:
# Now, Print those records
for i in agg_result: 
    print(i)

{'_id': 'Leo', 'Total Subject': 3}
{'_id': 'Sid', 'Total Subject': 3}


# Group by SUM of score

### Calculating the total score based on user

In [41]:
### Calculating the average score based on user
agg_result = collection.aggregate([
   {
      "$group": 
               {"_id": '$user', 
                "Total Marks": {"$sum": "$score"}}  #Here, we are calculationg the SUM of score based on the user which are present in collection
   }
])

In [42]:
# Now, Print those records (Out of 300 marks from 3 subjects they got: )
for i in agg_result: 
    print(i)

{'_id': 'Leo', 'Total Marks': 235}
{'_id': 'Sid', 'Total Marks': 250}


# B. Group by AVG of score

### Calculating the average score based on user

In [43]:
agg_result = collection.aggregate([
   {
      "$group": 
               {"_id": '$user', 
                "Students ave score": {"$avg": "$score"}}  #Here, we are calculationg the SUM of score based on the user which are present in collection
   }
])

In [44]:
# Now, Print those records (Out of 300 marks from 3 subjects they got: )
for i in agg_result: 
    print(i)

{'_id': 'Leo', 'Students ave score': 78.33333333333333}
{'_id': 'Sid', 'Students ave score': 83.33333333333333}


# DataTime analysis:

In [13]:
import datetime as datetime

In [14]:
### Create a new collection
data=[  { "_id" : 1, "item" : "abc", "price" : 10, "quantity" : 2, "date" : datetime.datetime.utcnow()},
        { "_id" : 2, "item" : "jkl", "price" : 20, "quantity" : 1, "date" : datetime.datetime.utcnow()},
        { "_id" : 3, "item" : "xyz", "price" : 5, "quantity" : 5, "date" : datetime.datetime.utcnow()},
        { "_id" : 4, "item" : "abc", "price" : 10, "quantity" : 10, "date" : datetime.datetime.utcnow()},
        { "_id" : 5, "item" : "xyz", "price" : 5, "quantity" : 10, "date" :datetime.datetime.utcnow()}
     ]

In [15]:
data

[{'_id': 1,
  'item': 'abc',
  'price': 10,
  'quantity': 2,
  'date': datetime.datetime(2023, 1, 2, 6, 15, 8, 448078)},
 {'_id': 2,
  'item': 'jkl',
  'price': 20,
  'quantity': 1,
  'date': datetime.datetime(2023, 1, 2, 6, 15, 8, 448078)},
 {'_id': 3,
  'item': 'xyz',
  'price': 5,
  'quantity': 5,
  'date': datetime.datetime(2023, 1, 2, 6, 15, 8, 448078)},
 {'_id': 4,
  'item': 'abc',
  'price': 10,
  'quantity': 10,
  'date': datetime.datetime(2023, 1, 2, 6, 15, 8, 448078)},
 {'_id': 5,
  'item': 'xyz',
  'price': 5,
  'quantity': 10,
  'date': datetime.datetime(2023, 1, 2, 6, 15, 8, 448078)}]

In [16]:
# Now, Create a collection 'stores' and store the above data in it!
mycollection = mydatabase['stores']  
mycollection.insert_many(data)

### Find Overall Avg Price and Avg Quantity (this happens based on date) 

### To find avg of all the quantities (we need to Multiply Price with Qty) i.e we want a multiply of 10 * 2 = 20 ...so on

In [45]:
### Find Overall Avg Price and Avg Quantity (this happens based on date) i.e we want a multiply of 10 * 2 = 20 ...
### Calculating the average quantity And Average Price
agg_result=mycollection.aggregate([
   {
      "$group": {
         "_id": '$item',      #Here, based on the item we have group it 
         "avgAmount": {"$avg": {"$multiply": [ "$price", "$quantity" ]}},  #To find avg of all the quantities (we need to Multiply Price with Qty)
          "avgQuantity": { "$avg" : "$quantity" }
      }
   }
])

In [46]:
# Now, Print those records 
for i in agg_result: 
    print(i)

{'_id': 'abc', 'avgAmount': 60.0, 'avgQuantity': 6.0}
{'_id': 'xyz', 'avgAmount': 37.5, 'avgQuantity': 7.5}
{'_id': 'jkl', 'avgAmount': 20.0, 'avgQuantity': 1.0}


# C. $project

**Its like a SELECT 'NAME_OF_COLUMNS,..' in MySQL**

In [22]:
#data from documentation : https://www.mongodb.com/docs/manual/reference/operator/aggregation/project/#mongodb-pipeline-pipe.-project   (Always refer documentation)
data = [{
              "_id" : 1,
              "title": "abc123",
              "isbn": "0001122223334",
              "author": { "last": "zzz", "first": "aaa" },
              "copies": 5
            },
            {
              "_id" : 2,
              "title": "Baked Goods",
              "isbn": "9999999999999",
              "author": { "last": "xyz", "first": "abc", "middle": "" },
              "copies": 2
            },
            {
              "_id" : 3,
              "title": "Ice Cream Cakes",
              "isbn": "8888888888888",
              "author": { "last": "xyz", "first": "abc", "middle": "mmm" },
              "copies": 5
       }]

In [47]:
# Access collection of the database called as 'students' and Access database  

collection = mydatabase['Books']  

In [24]:
# Insert many documents inside the collection// Records in the form of JSON
collection.insert_many(data)

In [48]:
for row in collection.aggregate([ { "$project": { "title": 1,"isbn":1 }} ]):
    print(row)                                                  ##  like a SELECT 'NAME_OF_COLUMNS,..' in MySQL

{'_id': 1, 'title': 'abc123', 'isbn': '0001122223334'}
{'_id': 2, 'title': 'Baked Goods', 'isbn': '9999999999999'}
{'_id': 3, 'title': 'Ice Cream Cakes', 'isbn': '8888888888888'}
